# 科研基建与Python内功

> 目标： 搭建专业、高效的科研环境，掌握Python核心高级特性，为后续深度学习打下坚实基础。

## Part1:Git基础命令 (clone, add, commit, push, pull), 分支管理 (branch, checkout, merge)。

In [ ]:
git init
git branch -M main
git add .
git commit -m "#描述改动内容"
(git remote add origin <url>.git) #第一次连接项目和仓库，如果本地项目是从仓库clone下来的，默认直接连接
git push -u origin main #除了第一次push要加-u，后面都不用
git pull origin main #把远程仓库的改动（合作者的改动）pull到本地，每次开始前都先pull一次最新的避免冲突

In [ ]:
# 查看所有分支（* 表示当前所在分支）
git branch

# 创建新分支（比如叫 "experiment-svm"）
git branch experiment-svm

# 切换到新分支（见下条 checkout）
git checkout experiment-svm

# 也可以一步创建并切换：
git checkout -b experiment-svm

# 实验成功，要合并改动了，先切换回 main 分支
git checkout main

# 把 experiment-svm 的改动合并进来
git merge experiment-svm

git branch -d experiment-failed  # 删除本地分支
git push origin --delete experiment-failed  # 删除远程分支

In [ ]:
#解决冲突
'''看到
<<<<<<< HEAD
accuracy = 0.95  # 你写的
=======
accuracy = 0.92  # 同学写的
>>>>>>> experiment-svm'''

#手动编辑文件，保留你想要的版本（或融合两者）
#删除 <<<<<<<, =======, >>>>>>> 这些标记
git add model.py
git commit -m "resolve merge conflict"

## Part2:Docker核心概念 (镜像、容器)

In [ ]:
#docker 和 github 的区别在于，docker可以创建dockerfile，确保实验环境的一致性

In [ ]:
#场景1：复现别人论文
docker pull author/paper-env:latest #作者提供 Docker 镜像
docker run -it author/paper-env bash #本地一键启动容器
#用 --gpus all 让容器使用 GPU（需安装 nvidia-docker）
#用 -p 8000:8000 映射端口，比如跑 JupyterLab 或 TensorBoard
cd /project
python train.py #直接复现实验，如果作者不靠谱的话需要自己用-v把代码挂载到容器里

In [ ]:
#场景2:管理自己的多个实验环境
#训练前打镜像标签 → myproject:v1-train, myproject:v2-finetune
docker build -t myproject:exp1 .#构建镜像1
docker build -t myproject:exp2 .#构建镜像2
#然后本地会出现
REPOSITORY   TAG     IMAGE ID
myproject    exp1    a1b2c3d
myproject    exp2    e4f5g6h
#实际项目常见目录
cd exp1 && docker build -t mypaper:exp1 .
cd ../exp2 && docker build -t mypaper:exp2 .
'''
my-paper/
├── exp1/
│   ├── Dockerfile          # FROM pytorch:1.10
│   └── train.py
├── exp2/
│   ├── Dockerfile          # FROM pytorch:2.0
│   └── train.py
└── data/                   # 共享数据目录
'''

In [ ]:
#场景3：交付可复现成果
#写完论文，把代码+环境打包成镜像 → 上传 Docker Hub
#审稿人 docker run 就能跑你的实验 → 复现性满分！

In [ ]:
docker pull ubuntu:20.04 #从仓库拉取镜像
docker images #查看本地镜像
docker run -it ubuntu:20.04 bash #本地运行容器，-it表示交互式终端
docker ps #查看正在运行的容器
docker ps -a #查看所有容器，包括已经停止的
docker exec -it <容器名> bash #进入运行中的容器
docker stop <容器名> #终止容器
docker rm <容器名>#删除容器
docker rmi <镜像名>#删除镜像
docker commit <容器名> my-new-image #保存容器为新镜像，固定实验环境
docker run -v /host/path:/container/path #挂载本地目录，让容器读写你本地的代码/数据，持久化实验数据

In [ ]:
#每个项目建一个 Dockerfile（镜像制作说明书）→ 环境可复现
FROM ubuntu:20.04
RUN apt update && apt install -y python3 python3-pip
COPY requirements.txt .
RUN pip3 install -r requirements.txt
COPY . /app
WORKDIR /app
CMD ["python3", "train.py"]

In [ ]:
#生成一个叫 my-experiment 的镜像，别人可以直接用！
#. → 表示“当前目录”是构建上下文（context）,对应的dockerfile必需在当前目录里
docker build -t my-experiment .
#上传镜像到 Docker Hub 
docker login
docker tag myproject:exp1 zhangsan/myproject:exp1 #tag命名规范<用户名>/<仓库名>:<标签>
docker tag myproject:exp2 zhangsan/myproject:exp2
docker push zhangsan/myproject:exp1
docker push zhangsan/myproject:exp2

In [ ]:
## 论文附录写复现步骤
docker pull yourname/project:final
docker run -it -v $(pwd)/data:/data yourname/project:final bash
cd /project && python eval.py

In [ ]:
# 科研常用完整步骤
# 构建镜像（在 Dockerfile 所在目录）
docker build -t myexp:v1 .

# 启动容器 + 挂载代码 + 挂载数据 + 使用GPU
docker run -it \
  --gpus all \
  -v $(pwd):/workspace \
  -v /data/datasets:/data \
  --name my-running-exp \
  myexp:v1 \
  bash

# 进入已运行的容器（调试用）
docker exec -it my-running-exp bash

# 推送到 Docker Hub
docker tag myexp:v1 yourname/myexp:v1
docker push yourname/myexp:v1

## Part3:W&B: 注册账号，学习 wandb.init(), wandb.log() 记录标量

In [ ]:
#终端
pip install wanb
wanb login #需要copy apikey进去

In [ ]:
#初始化一次实验
#调用 wandb.init() 后，W&B 会在后台创建一个“实验运行（Run）”，并分配唯一 ID
#project：一个论文/课题一个项目，所有相关实验都在这里
#name：比如 "lr-0.01-batch64", "with-data-aug", "final-submission"
#config：记录所有超参数，后期可在网页端筛选、对比实验！
import wandb

wandb.init(
    project="my-awesome-paper",   # 项目名（必填！）
    name="exp-svm-linear",        # 实验名（可选，方便你识别）
    config={                      # 超参数（可选，但强烈建议！）
        "learning_rate": 0.001,
        "batch_size": 32,
        "model": "ResNet18",
        "epochs": 100,
    }
)

In [ ]:
# 记录标量
# 每训练完一个 epoch，你在笔记本上画一个点：横轴 epoch，纵轴 loss 
for epoch in range(100):
    train_loss = train_one_epoch(...)  # 你的训练函数
    val_acc = validate(...)            # 你的验证函数

    # ✅ 重点！记录标量到 W&B
    wandb.log({
        "train_loss": train_loss,
        "val_acc": val_acc,
        "epoch": epoch
    })

In [ ]:
# pytorch科研完整模版
# 每个实验都开一个 run：不同超参数、不同模型结构，都用 wandb.init() 分开
# config 里放所有超参数：后期对比实验时神器！
# log 频率不用太高：每个 epoch log 一次足够，避免数据爆炸
# 起好实验名（name）：比如 "bs128-lr0.001-aug"，方便后期查找
import wandb
import torch
import torch.nn as nn
from torchvision import models

# ========== 1. 初始化 W&B ==========
wandb.init(
    project="image-classification-paper",
    name="resnet18-lr0.001",
    config={
        "learning_rate": 0.001,
        "batch_size": 32,
        "architecture": "ResNet18",
        "dataset": "CIFAR-10",
        "epochs": 50,
    }
)

# ========== 2. 伪代码：模型、数据、优化器 ==========
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.resnet18(pretrained=False, num_classes=10).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=wandb.config.learning_rate)

# ========== 3. 训练循环 ==========
for epoch in range(wandb.config.epochs):
    model.train()
    train_loss = 0.0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    # 验证（伪代码）
    model.eval()
    val_acc = evaluate(model, val_loader)  # 假设你有这个函数

    # ========== 4. 记录到 W&B！==========
    wandb.log({
        "train_loss": train_loss / len(train_loader),
        "val_accuracy": val_acc,
        "epoch": epoch
    })

    print(f"Epoch {epoch}: Loss={train_loss:.4f}, Acc={val_acc:.4f}")

# ========== 5. 结束（可选）==========
wandb.finish()

In [ ]:
wandb.log({"val_acc": val_acc}, step=epoch)  # 显式指定 x 轴值

current_lr = optimizer.param_groups[0]['lr']
wandb.log({"learning_rate": current_lr}) #记录学习率

wandb.log({
    "loss/train": train_loss,
    "loss/val": val_loss,
    "accuracy/train": train_acc,
    "accuracy/val": val_acc,
}) # 记录多个曲线在同一张图（用相同 key 前缀），W&B 会自动分组，生成 loss 和 accuracy 两个图表。

In [9]:
'''使用Docker容器，在本地运行一个简单的PyTorch脚本（例如，随机生成两个Tensor并相加），并将计算结果（如Tensor的和）通过W&B记录到云端。'''

'使用Docker容器，在本地运行一个简单的PyTorch脚本（例如，随机生成两个Tensor并相加），并将计算结果（如Tensor的和）通过W&B记录到云端。'

## Part4:Python常用工具（生成器与装饰器，上下文管理器与常用库）

In [2]:
#生成器 (yield): 理解其惰性求值和内存优势，对比列表。

In [3]:
#装饰器: 理解闭包，掌握无参和有参装饰器的写法。

In [8]:
'''
编写一个 @timer 装饰器，用于测量任何函数的执行时间。
编写一个 @memory_profiler 装饰器（可以使用 torch.cuda.memory_allocated()），
用于测量函数执行前后的GPU显存变化。将这两个装饰器应用到第1周的PyTorch脚本中，
并通过W&B记录时间和显存数据。
'''

'\n编写一个 @timer 装饰器，用于测量任何函数的执行时间。\n编写一个 @memory_profiler 装饰器（可以使用 torch.cuda.memory_allocated()），\n用于测量函数执行前后的GPU显存变化。将这两个装饰器应用到第1周的PyTorch脚本中，\n并通过W&B记录时间和显存数据。\n'

In [10]:
#上下文管理器: 理解 __enter__ 和 __exit__，掌握 @contextmanager 装饰器。
#上下文管理器确保资源的“自动获取与释放”、“初始化与清理”，无论是否发生异常.避免忘记关闭文件，提升代码健壮性，科研中常用于：日志写入、模型保存、临时文件、GPU显存监控等
# 最常用的是with语句
with open('data.txt','r') as f: # 文件会自动关闭，哪怕中间报错
    content = f.read

In [11]:
#自定义上下文管理器
#法1:使用类实现 __enter__ 和 __exit__
import time
class Timer:
    def __enter__(self): #当进入 with 语句块时，__enter__ 方法被自动调用。
        self.start = time.time() #记录当前时间（进入 with 块的时间）
        return self # 在 with Timer() as t: 中，变量 t 就会绑定到这个 Timer 实例
    def __exit__(self,exc_type,exc_val,exc_tb): #当 with 语句块执行完毕（或发生异常）时，__exit__ 方法被自动调用。后面3个参数分别捕捉异常类型，值，traceback信息
        print(f"耗时{time.time()-self.start:.4f}秒")
        return False #不抑制异常。如果 with 块中发生异常，会继续向外抛出。
with Timer() as t: #创建 Timer 实例，调用其 __enter__ 方法，返回值赋给 t
    time.sleep(1) #模拟耗时操作，暂停 1 秒。
#退出 with 块后，自动调用 __exit__，打印耗时。

耗时1.0014秒


In [ ]:
#法2:使用 contextlib.contextmanager 装饰器（推荐）
from contextlib import contextmanager
import torch

@contextmanager
def gpu_memory_monitor(device=0):
    print(f"[开始] GPU {device} 显存: {torch.cuda.memory_allocated(device)//1024**2} MB")
    yield
    print(f"[结束] GPU {device} 显存: {torch.cuda.memory_allocated(device)//1024**2} MB")

# 使用
with gpu_memory_monitor():
    model = MyBigModel().cuda()
    output = model(input_tensor)

In [22]:
# python的类复习强化
class Animal:
    def __init__(self,name):
        self.name = name
    def speak(self):
        pass

class Dog(Animal):
    def speak(self):
        return "woof"

class Robot(Animal):
    def operate(self):
        return f"I'm {self.name}, a robot."

class DogRobo(Dog,Robot):
    def __init__(self,name):
        Dog.__init__(self,name)
        Robot.__init__(self,name)
robodog = DogRobo('Roy')
print(robodog.name)
print(robodog.speak())
print(robodog.operate())

Roy
woof
I'm Roy, a robot.


In [11]:
#常用库: os, glob (文件路径操作), json (配置文件读写), tqdm (进度条)。

In [ ]:
# 文件i/o操作，基础读写
# 写入文本
with open('output.txt', 'w', encoding='utf-8') as f:
    f.write("Hello World\n")
    f.writelines(["Line 1\n", "Line 2\n"])

# 读取文本
with open('output.txt', 'r', encoding='utf-8') as f:
    content = f.read()        # 全部内容
    lines = f.readlines()     # 按行读取列表

# 逐行读取大文件（省内存）
with open('large_file.txt') as f:
    for line in f:
        process(line)

In [ ]:
# 二进制读写
# 保存模型权重（PyTorch 示例）
torch.save(model.state_dict(), 'model.pth')

# 加载
model.load_state_dict(torch.load('model.pth'))

# 或者用 pickle（通用对象序列化）
import pickle
with open('config.pkl', 'wb') as f:
    pickle.dump(config_dict, f)

with open('config.pkl', 'rb') as f:
    config = pickle.load(f)

In [ ]:
# os库，操作系统接口
import os

# 路径操作（推荐用 os.path 或 pathlib）
path = os.path.join("data", "train", "img_001.jpg")
basename = os.path.basename(path)   # "img_001.jpg"
dirname = os.path.dirname(path)     # "data/train"

# 创建目录（避免重复创建报错）
os.makedirs("checkpoints", exist_ok=True)

# 列出目录内容
files = os.listdir("data/")

# 环境变量（常用于配置路径、密钥）
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'

In [ ]:
#json，配置文件/实验记录
import json

# 写入配置
config = {
    "lr": 1e-4,
    "batch_size": 32,
    "epochs": 100
}

with open('config.json', 'w') as f:
    json.dump(config, f, indent=2)  # indent 让格式美观

# 读取配置
with open('config.json', 'r') as f:
    loaded_config = json.load(f)

print(loaded_config["lr"])  # 0.0001

In [ ]:
# glob文件名模式匹配
import glob

# 匹配所有 jpg 文件
image_files = glob.glob("data/train/*.jpg")

# 递归匹配子目录中的 txt 文件
txt_files = glob.glob("data/**/*.txt", recursive=True)

# 在数据加载器中常用：
train_images = sorted(glob.glob("dataset/*/images/*.png"))
train_labels = [f.replace('/images/', '/labels/') for f in train_images]

In [ ]:
# tqdm进度条神器
from tqdm import tqdm
import time

# 包装任何可迭代对象
for i in tqdm(range(1000)):
    time.sleep(0.01)  # 模拟耗时操作

# 在 DataLoader 中使用（PyTorch）
for batch in tqdm(train_loader, desc="Epoch 1"):
    train_step(batch)

# 手动更新（用于复杂循环）
pbar = tqdm(total=100)
for i in range(10):
    do_something()
    pbar.update(10)  # 每次更新10%
pbar.close()